In [1]:
import os
import numpy as np
import pandas as pd
import random
import glob
import datetime
import itertools
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import nbformat

from scipy.optimize import nnls

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Parameters and File Paths

In [2]:
SCORE_VAR = 'frac_alpha_leq_25pct'

#--- Local paths
EXPERIMENT = 'BLUEPRINT_B'
ROOT_DIR = '/analysis/gh-msun/'
PROJECT_SLUG = '2023_06_26_SRT_deconvolution_MS'
PROJECT_DIR = ROOT_DIR + f'projects/{PROJECT_SLUG}/output/'
DATA_DIR = PROJECT_DIR + f'/experiment/{EXPERIMENT}/'
SCORE_DIR = DATA_DIR + 'methyl_score/'

REGION_PATH = (
    PROJECT_DIR + \
    'reference/deconv_inhouse_v2.region_to_ref_celltype.tsv.gz'
)

# Reference matrix
ATLAS_PATH = (
    PROJECT_DIR + \
    'reference/deconv_inhouse_v2.atlas.tsv.gz'
)

# Methylation score matrix
SCORE_PATH = (
    SCORE_DIR + \
    'E1B_E18CD4_E18CD8_E18NK_E18MONO_E18NEUTRO/' + \
    'mix0_seed512070.tsv.gz'
)
    
# FILTER_COV = 20
# FILTER_COV_COEFF = 0.5

# # Sample annotations
# SAMPLE_PATH = DATA_DIR + ''
# # Methylation data
# SCORE_PATH = (
#     PROJECT_DIR + '/output/meth_summaries/buffycoat_meth_summaries_cg_count_geq_3_deconvolution_v2.v23_conv.with_cpg_index.tsv.gz'
# )

# COVERAGE_PATH = (
#     PROJECT_DIR + '/data/region_coverage_estimates.deconvolution_v2.v23_conv.tsv'
# )
# # Where to store graphics
# RESULTS_PATH = PROJECT_DIR + '/output/deconv/deconv_inhouse_v2_blueprint'

In [96]:
def get_file_paths(directory):

    list_paths = []

    for filename in os.listdir(directory):
        list_paths.append(os.path.abspath(os.path.join(directory, filename)))

    return(list_paths)


def compute_deconvolution_nnls(score_df_path, score_type, atlas, match=True):
    '''
    Run nonnegative least squares ||Ax-b||_2. 
    The solution x is the deconvolution of b.
    
    Reasoning for match=True:
    Note that for lower total read count for a mixture, there be regions that
    are missing in score_df. At 1M reads, this not a problem.
    
    score_df -- methylation score dataframe
    score_type -- hypo or hyper score: e.g. 'frac_alpha_leq_25pct'
    atlas -- atlas dataframe
    '''
    # load score df
    score_df = pd.read_csv(score_df_path, sep='\t')
    score_df.index = score_df.region_id
    
    b = score_df[score_type]
    A = atlas
    
    # match index between A and b
    if match:
        region_count_before = A.shape[0]
        A = A[A.index.isin(b.index)]
        region_count_after = A.shape[0]
        region_count_diff = region_count_before - region_count_after 
        print(f'Dropped: {region_count_diff} regions.')
    
    # sort the indices for A to match b indices
    A_sorted = A.loc[b.index, :]
    
    # run NNLS
    fit = nnls(A_sorted, b)
    x = pd.Series(fit[0], index=A_sorted.columns)
    
    return(x)


def compute_deconvolution_n_times(mixture_replicates_path, score_type, atlas, match=True):
    '''
    
    mixture_replicates_path -- path to a mixture (proportion) directory of replicates (e.g. ../E1B_E18CD4_E18CD8_E18NK_E18MONO_E18NEUTRO/)
    output: pandas df
    
    '''
    # given path to mixture grab all paths to mixture replicates
    list_mixture_dir_paths = get_file_paths(mixture_replicates_path)
    
    # run deconvolution for each replicate
    results = []
    samples_name = []
    for path in list_mixture_dir_paths:
        deconv = compute_deconvolution_nnls(score_df_path=path, 
                                           atlas=atlas, 
                                           score_type=score_type, 
                                           match=match)
        results.append(deconv)
    df = pd.concat(results, axis=1)
    
    return df 


def compute_deconvolution_methyl_score_dir(path_to_methyl_score_dir, score_type, atlas, match=True):
    '''
    '''
    # grab all file paths in methyl_score directory
    list_methyl_score_dir = get_file_paths(path_to_methyl_score_dir)
    
    # run deconvolution on each mixture proportion
    results = []
    for path in list_methyl_score_dir:
        df = compute_deconvolution_n_times(mixture_replicates_path=path, 
                                               score_type=score_type, 
                                               atlas=atlas, 
                                               match=match)
        results.append(df)
    
    return(results)
    

def mse():
    pass

### ATLAS

In [97]:
atlas = pd.read_csv(ATLAS_PATH, sep='\t',index_col=None)
atlas = atlas.set_index('region_id')
atlas.columns.name = 'ref_celltype'
atlas.shape

(280, 9)

In [98]:
ref_region_df = pd.read_csv(REGION_PATH, sep='\t')
ref_region_df.shape[0]

280

In [99]:
atlas = atlas.loc[ref_region_df['region_id'], :].copy()
atlas.shape

(280, 9)

## Deconvolution

In [102]:
testing = compute_deconvolution_methyl_score_dir(path_to_methyl_score_dir='/analysis/gh-msun/projects/2023_06_26_SRT_deconvolution_MS/output/experiment/BLUEPRINT_B/methyl_score/', 
                                                  score_type=SCORE_VAR, 
                                                  atlas=atlas, 
                                                  match=False)



In [103]:
testing[2]

,0,1,2,3,4,5,6,7,8,9
ref_celltype,,,,,,,,,,
immune_b,0.000897,0.000000,0.001195,0.000628,0.000046,0.001387,0.000878,0.000000,0.001288,0.000269
immune_t,0.371986,0.379268,0.378020,0.383450,0.388740,0.392394,0.393120,0.394021,0.389623,0.386005
immune_nk,0.195111,0.192918,0.196866,0.191918,0.194773,0.192806,0.194427,0.197294,0.198425,0.197637
immune_dend_plus_macro_plus_mono,0.222617,0.205609,0.210477,0.206698,0.194845,0.224213,0.217259,0.192153,0.204735,0.201214
immune_eosi,0.000000,0.000000,0.000000,0.000000,0.002384,0.000000,0.000000,0.000000,0.000000,0.000000
immune_neutro,0.199549,0.188972,0.196473,0.191006,0.187598,0.189004,0.196511,0.199537,0.191798,0.205025
immune_eryth,0.000722,0.000000,0.001624,0.000000,0.000000,0.002986,0.000328,0.000306,0.000000,0.000000
immune_mega,0.000000,0.000000,0.000314,0.000000,0.000000,0.002626,0.000000,0.000000,0.000000,0.000000
eryth_prog,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
